<a href="https://www.kaggle.com/code/racemhamdy/yolo-v9?scriptVersionId=174858245" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-dataset/dyolo/dyolo.yaml
/kaggle/input/text-dataset/dyolo/valid/labels/26cecb6e5b984c20.txt
/kaggle/input/text-dataset/dyolo/valid/labels/3ea969425b1c88bd.txt
/kaggle/input/text-dataset/dyolo/valid/labels/4e0277825c80fed8.txt
/kaggle/input/text-dataset/dyolo/valid/labels/78f50574b5cf4a22.txt
/kaggle/input/text-dataset/dyolo/valid/labels/96fb2a4a66702374.txt
/kaggle/input/text-dataset/dyolo/valid/labels/4f470a000d2ce41e.txt
/kaggle/input/text-dataset/dyolo/valid/labels/9d7a8cdb3953fe37.txt
/kaggle/input/text-dataset/dyolo/valid/labels/79e130fb92d0d6cc.txt
/kaggle/input/text-dataset/dyolo/valid/labels/312e159cadc95211.txt
/kaggle/input/text-dataset/dyolo/valid/labels/6b9a238096d21ac9.txt
/kaggle/input/text-dataset/dyolo/valid/labels/3a7caa6122f867ec.txt
/kaggle/input/text-dataset/dyolo/valid/labels/08d21add8f62215c.txt
/kaggle/input/text-dataset/dyolo/valid/labels/372e685273b705df.txt
/kaggle/input/text-dataset/dyolo/valid/labels/00bb9343208cd7f0.txt
/kaggle/input/text

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (668/668), done.
remote: Compressing objects: 100% (405/405), done.
remote: Total 668 (delta 271), reused 577 (delta 246), pack-reused 0
Receiving objects: 100% (668/668), 3.23 MiB | 30.34 MiB/s, done.
Resolving deltas: 100% (271/271), done.
/kaggle/working/yolov9


In [4]:
!wget -P /kaggle/working/yolov9/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P /kaggle/working/yolov9/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P /kaggle/working/yolov9/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P /kaggle/working/yolov9/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [5]:
!ls -la /kaggle/working/yolov9/weights

total 402440
drwxr-xr-x  2 root root      4096 Apr 27 15:48 .
drwxr-xr-x 13 root root      4096 Apr 27 15:48 ..
-rw-r--r--  1 root root  51508261 Feb 18 12:36 gelan-c.pt
-rw-r--r--  1 root root 117203713 Feb 18 12:36 gelan-e.pt
-rw-r--r--  1 root root 103153312 Feb 18 12:36 yolov9-c.pt
-rw-r--r--  1 root root 140217688 Feb 18 12:36 yolov9-e.pt


In [11]:
import shutil

# Define the directory path to be deleted
directory_to_delete = "/kaggle/working/yolov9.zip"

# Attempt to delete the directory
try:
    shutil.rmtree(directory_to_delete)
    print(f"Directory '{directory_to_delete}' deleted successfully.")
except OSError as e:
    print(f"Error: {directory_to_delete} : {e.strerror}")


Error: /kaggle/working/yolov9.zip : Not a directory


In [9]:
import yaml

# Read content from dyolo.yaml
with open('/kaggle/input/text-dataset/dyolo/dyolo.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Modify the content
data['names'] = ['paragraph', 'line', 'word']
data['test'] = '/kaggle/input/text-dataset/dyolo/test/images'
data['train'] = '/kaggle/input/text-dataset/dyolo/train/images'
data['val'] = '/kaggle/input/text-dataset/dyolo/valid/images'

# Write the modified content to new_data.yaml in the /kaggle/working/ directory
with open('/kaggle/working/new_data.yaml', 'w') as file:
    yaml.dump(data, file)


In [10]:
import yaml

# Define the file path
file_path = "/kaggle/working/new_data.yaml"

# Read the YAML file
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# Print the content
print("Content of new_data.yaml:")
print(yaml.dump(data))


Content of new_data.yaml:
names:
- paragraph
- line
- word
nc: 3
test: /kaggle/input/text-dataset/dyolo/test/images
train: /kaggle/input/text-dataset/dyolo/train/images
val: /kaggle/input/text-dataset/dyolo/valid/images



In [4]:
%cd /kaggle/working/yolov9

!python train.py \
--batch 8 --epochs 50 --img 416 --device 0 --min-items 0 --close-mosaic 15 \
--data /kaggle/working/new_data.yaml \
--weights /kaggle/working/yolov9/weights/gelan-e.pt \
--cfg models/detect/gelan-e.yaml \
--hyp hyp.scratch-high.yaml


/kaggle/working/yolov9
2024-04-28 00:13:37.957291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 00:13:37.957346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 00:13:37.958993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=/kaggle/working/yolov9/weights/gelan-e.pt, cfg=models/detect/gelan-e.yaml, data=/kaggle/working/new_data.yaml, hyp=hyp.scratch-hi

In [9]:
%cd /kaggle/working/yolov9

!python train.py \
--batch 8 --epochs 100 --img 416 --device 0 --min-items 0 --close-mosaic 15 \
--data /kaggle/working/new_data.yaml \
--weights /kaggle/working/yolov9/runs/train/exp/weights/last.pt \
--cfg models/detect/gelan-e.yaml \
--hyp hyp.scratch-high.yaml


/kaggle/working/yolov9
^C
Traceback (most recent call last):
  File "/kaggle/working/yolov9/train.py", line 25, in <module>
    import val as validate  # for end-of-epoch mAP
  File "/kaggle/working/yolov9/val.py", line 17, in <module>
    from models.common import DetectMultiBackend
  File "/kaggle/working/yolov9/models/common.py", line 26, in <module>
    from utils.dataloaders import exif_transpose, letterbox
  File "/kaggle/working/yolov9/utils/dataloaders.py", line 20, in <module>
    import torchvision
  File "/opt/conda/lib/python3.10/site-packages/torchvision/__init__.py", line 6, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils
  File "/opt/conda/lib/python3.10/site-packages/torchvision/models/__init__.py", line 2, in <module>
    from .convnext import *
  File "/opt/conda/lib/python3.10/site-packages/torchvision/models/convnext.py", line 8, in <module>
    from ..ops.misc import Conv2dNormActivation, Permute
  File "/op

KeyboardInterrupt: 

In [7]:
%cd /kaggle/working
!zip -r file.zip /kaggle/working



/kaggle/working
  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/yolov9/ (stored 0%)
  adding: kaggle/working/yolov9/train_triple.py (deflated 70%)
  adding: kaggle/working/yolov9/.git/ (stored 0%)
  adding: kaggle/working/yolov9/.git/config (deflated 33%)
  adding: kaggle/working/yolov9/.git/hooks/ (stored 0%)
  adding: kaggle/working/yolov9/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: kaggle/working/yolov9/.git/hooks/pre-push.sample (deflated 50%)
  adding: kaggle/working/yolov9/.git/hooks/pre-receive.sample (deflated 40%)
  adding: kaggle/working/yolov9/.git/hooks/fsmonitor-watchman.sample (deflated 52%)
  adding: kaggle/working/yolov9/.git/hooks/pre-commit.sample (deflated 45%)
  adding: kaggle/working/yolov9/.git/hooks/post-update.sample (deflated 27%)
  adding: kaggle/working/yolov9/.git/hooks/applypatch-msg.sample (deflated 42%)
  adding: kaggle/working/yolov9/.git/hooks/update.sample (deflated 68%)
  adding: kaggle/working/yolov9/.git/hooks/pre-reba